# Funding Successfull Projects on Kickstarter
*Resources: https://www.kaggle.com/codename007/funding-successful-projects

*Data: https://www.kaggle.com/codename007/funding-successful-projects/data

# Objective 

* Kickstarter is a community of more than 10 million people comprising of creative, tech enthusiasts who help in bringing creative project to life. Till now, more than $3 billion dollars have been contributed by the members in fueling creative projects. The projects can be literally anything – a device, a game, an app, a film etc.

* Kickstarter works on all or nothing basis i.e., if a project doesn’t meet its goal, the project owner gets nothing. 
* For example: if a project’s goal is $500. Even if it gets funded till $499, the project won’t be a success.

* In this challange, we will try to predict whether a project will get successfully funded or not.

# Dataset:
* For this project we will be using the dataset obtained from Kaggle and is in the link 
* https://www.kaggle.com/codename007/funding-successful-projects/data

In [4]:
%sh
# Right click on Kaggle Download button to get link information
# Next, place on a shared drive (below GSU OneDrive), and get link to the shared drive
# Placed the link below:
wget "https://mygsu-my.sharepoint.com/personal/wrobinson_gsu_edu/_layouts/15/guestaccess.aspx?docid=0e3b03f3c7df34344a9db478adce1e984&authkey=ARnwjDE8SCQ3TMRofgoIE2g&e=1883debab986481889ef02ea44d76f2a"  -O kickstarter.zip; unzip kickstarter.zip

In [5]:
%sh
ls /databricks/driver

In [6]:
train_df = spark.read.load("file:/databricks/driver/train.csv",\
                    format='com.databricks.spark.csv',\
                    header='true',\
                    mode = 'DROPMALFORMED',\
                    inferSchema='true')

train_df.printSchema()

In [7]:
display(train_df)

In [8]:
train_df.count()

In [9]:
test_df = spark.read.load("file:/databricks/driver/test.csv",\
                    format='com.databricks.spark.csv',\
                    header='true',\
                    mode = 'DROPMALFORMED',\
                    inferSchema='true')

test_df.printSchema()

In [10]:
display(test_df)

In [11]:
test_df.count()

# DATA Transforming and Cleaning
* Converting unixtime into YYYY-MM--DD HH:MM:SS
* Removing null rows

In [13]:
from pyspark.sql.types import *
from pyspark.sql.functions import to_date
import datetime, time
from pyspark.sql.functions import UserDefinedFunction

def toTimeStamp(s):
  # If you have milliseconds then divide by 1000.0
  if (s):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(long(s)/1.0))
  else:
    return s
toTimeStamp_UDF = UserDefinedFunction(toTimeStamp, StringType())

# Converting column data types
df = train_df
df = df.withColumn("goal", df["goal"].cast(IntegerType()))
df = df.withColumn("backers_count", df["backers_count"].cast(IntegerType()))
df = df.withColumn("final_status", df["final_status"].cast(IntegerType()))

# Readable dates...
df = df.withColumn("dealineDT", toTimeStamp_UDF(df.deadline))
df = df.withColumn("created_atDT", toTimeStamp_UDF(df.created_at))
df = df.withColumn("launched_atDT", toTimeStamp_UDF(df.launched_at))
df = df.withColumn("state_changed_atDT", toTimeStamp_UDF(df.state_changed_at))


display(df)

In [15]:
display(train_df)

In [16]:
from pyspark.sql.types import *
from pyspark.sql.functions import to_date
import datetime, time
from pyspark.sql.functions import UserDefinedFunction

def toTimeStamp(s):
  # If you have milliseconds then divide by 1000.0
  if (s):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(long(s)/1.0))
  else:
    return s
toTimeStamp_UDF = UserDefinedFunction(toTimeStamp, StringType())

# Converting column data types
df1 = test_df
df1 = df1.withColumn("goal", df1["goal"].cast(IntegerType()))

# Readable dates...
df1 = df1.withColumn("dealineDT", toTimeStamp_UDF(df1.deadline))
df1 = df1.withColumn("state_changed_atDT", toTimeStamp_UDF(df1.state_changed_at))
df1 = df1.withColumn("created_atDT", toTimeStamp_UDF(df1.created_at))
df1 = df1.withColumn("launched_atDT", toTimeStamp_UDF(df1.launched_at))


display(df1)

In [17]:
trainCleaned = df.filter(df.project_id.isNotNull() & df.name.isNotNull() & df.desc.isNotNull())
display(trainCleaned)

In [18]:
trainCleaned.count()

In [19]:
testCleaned = df1.filter(df1.project_id.isNotNull() & df1.desc.isNotNull())
display(testCleaned)

In [20]:
testCleaned.count()

#Pattern Discovery and Model Evaluation

In [22]:
trainCleaned = df.filter(df.project_id.isNotNull() & df.name.isNotNull() & df.desc.isNotNull())
display(trainCleaned)


In [23]:
* As you can see only Only 30% of Projects are granted.

In [24]:
trainCleaned = df.filter(df.project_id.isNotNull() & df.name.isNotNull() & df.desc.isNotNull())
display(trainCleaned)
#Below is the graph for disable communication

In [25]:
trainCleaned.groupBy('disable_communication').count().show()

In [26]:
trainCleaned.registerTempTable("Train_temp")

In [27]:
%sql
SELECT final_status,
sum(CASE WHEN goal>= 0 AND goal < 10000 THEN 1 END) AS  G10K,
sum(CASE WHEN goal>= 10000 AND goal < 50000 THEN 1 END) AS G10Kto50K,
sum(CASE WHEN goal>= 50000 AND goal < 100000 THEN 1 END) AS G50Kto100K,
sum(CASE WHEN goal>= 100000 AND goal < 1000000 THEN 1 END) AS G100Kto1M,
sum(CASE WHEN goal>= 1000000 THEN 1 END) AS G1M
FROM Train_temp as Goalgroups
group by final_status

In [28]:
%sql
select log(goal) as log_goal from Train_temp

In [29]:
%sql
select backers_count, final_status, log(goal) as log_goal from Train_temp

* Average Goal value when Funding is not approved

In [31]:
%sql
select avg(goal) from Train_temp where final_status =0 group by final_status


* Average Goal value when Funding is approved

In [33]:
%sql
select avg(goal) from Train_temp where final_status =1 group by final_status

* The mean is high for successful Funding. And For 100% Successful funding "disable_communication" is False.

In [35]:
%sql
select final_status, goal, disable_communication from Train_temp

In [36]:
%sql
select country, final_status from Train_temp

In [37]:
%sql
select currency, country,final_status from Train_temp

* As you can see Most of the the application for Funding come from US, GB, Canada and Austrailia.

In [39]:
%sql
select backers_count,final_status from Train_temp

#We can easily conclude that backers_count is high for successfully funded projects.